In [4]:
from xml.etree.ElementPath import prepare_predicate
import monai
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import ImageGrid
import skimage.io as io
import os
from skimage.transform import resize
import torchio as tio
import pandas as pd 
import sys
import json
import pathlib
import SimpleITK as sitk
from tqdm.notebook import tqdm
main_dir = os.path.join("/ssd_Samsung870_2T/docker_submission",'nnUNet/nnunet')
os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
os.environ['RESULTS_FOLDER'] = os.path.join(main_dir,'nnUNet_trained_models')

In [5]:
os.listdir(os.path.join(main_dir,'nnUNet_raw_data_base'))

['nnUNet_cropped_data', 'nnUNet_raw_data']

In [10]:
# !nnUNet_find_best_configuration -m 3d_fullres -tr nnUNetTrainerV2_insaneDA -t 119



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

running missing postprocessing for Task119_final_model and model 3d_fullres
^C
Process ForkPoolWorker-16:
Process ForkPoolWorker-12:
Process ForkPoolWorker-15:
Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/ssd_Samsung870_2T/.miniforge3/envs/nnunet/bin/nnUNet_find_best_configuration", line 33, in <module>
    sys.exit(load_entry_point('nnunet', 'console_scripts', 'nnUNet_find_best_configuration')())
  File "/ssd_Samsung870_2T/nnUNet/nnunet/evaluation/model_selection/figure_out_what_to_submit.py", line 144, in main
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-14:
Process ForkPo

In [ ]:
os.system('nnUNet_predict -i /ssd_Samsung870_2T/docker_submission/input_input/ -o /ssd_Samsung870_2T/docker_submission/output_output_hou/ -t 119 -tr nnUNetTrainerV2 -m 3d_fullres --save_npz')
os.system('nnUNet_predict -i /ssd_Samsung870_2T/docker_submission/input_input/ -o /ssd_Samsung870_2T/docker_submission/output_output_shahad/ -t 119 -tr nnUNetTrainerV2_Loss_DiceCE_noSmooth -m 3d_fullres --save_npz')
os.system('nnUNet_predict -i /ssd_Samsung870_2T/docker_submission/input_input/ -o /ssd_Samsung870_2T/docker_submission/output_output_hussain/ -t 119 -tr nnUNetTrainerV2_insaneDA -m 3d_fullres --save_npz')


In [11]:
os.system('nnUNet_ensemble -f /ssd_Samsung870_2T/docker_submission/output_output_hou/ /ssd_Samsung870_2T/docker_submission/output_output_shahad/ /ssd_Samsung870_2T/docker_submission/output_output_hussain/ -o /ssd_Samsung870_2T/docker_submission/output_output/ -pp /ssd_Samsung870_2T/docker_submission/nnUNet/nnunet/nnUNet_trained_models/nnUNet/postprocessing.json')



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_separate_z: None interpolation order: 3
no resampling necessary
force_se

0

In [16]:
os.system('rm -rf /ssd_Samsung870_2T/docker_submission/output_output/not_postprocessed')
os.system('rm -rf /ssd_Samsung870_2T/docker_submission/output_output/*.json')


INPUT_FOLDER = "/ssd_Samsung870_2T/docker_submission/output_output" #fix here
ORI_PATH = "/ssd_Samsung870_2T/docker_submission/input"
OUTPUT_FOLDER = "/ssd_Samsung870_2T/docker_submission/output" #fix here
os.makedirs(OUTPUT_FOLDER,exist_ok=True)

test_df = pd.read_csv("/ssd_Samsung870_2T/docker_submission/test_info.csv") #fix here

for i in os.listdir(INPUT_FOLDER): 

    k = i[:-7] +".nii.gz" #how it was saved in the test csv 
    #In the csv file, the sizes are saved as a string of list, so we need to convert it to list
    list_str = test_df.loc[test_df['img_files'] == k, 'resampling_size'].item() #pick the size for the img 
    target_size = json.loads(list_str)

    label = tio.LabelMap(os.path.join(INPUT_FOLDER,i))

    transform_1 = tio.CropOrPad(target_size)
    label_transform_1 = transform_1(label)

    #In the csv file, the spacings are saved as a string of tuple, so we need to convert it to tuple
    spacing_str = test_df.loc[test_df['img_files'] == k, 'org_img_spacing'].item() #pick the spacing for the img 
    target_spacing = eval(spacing_str)

    transform_2 = tio.Resample(target_spacing, image_interpolation='nearest', label_interpolation="nearest")
    label_transform_2 = transform_2(label_transform_1)

    s = i[:-12] + "_Label.nii.gz"
    label_transform_2.save(os.path.join(OUTPUT_FOLDER,s))


#Confirm label sizes match original test image sizes
for i in os.listdir(OUTPUT_FOLDER):

    label = tio.LabelMap(os.path.join(OUTPUT_FOLDER, i))
    label_size = list(label.shape[1:4])

    k = i[:21] +"_hrT2.nii.gz" #Remove "_Label.nii.gz" and add "_hrT2.nii.gz" for the testing images
    
    #In the csv file, the sizes are saved as a string of list, so we need to convert it to list
    list_str = test_df.loc[test_df['img_files'] == k, 'org_img_size'].item() #pick the size for the img 
    org_size = json.loads(list_str)

    if label_size == org_size: 
        continue 
    else: 
        transform = tio.CropOrPad(org_size)
        label_transform = transform(label)
        label_transform.save(os.path.join(OUTPUT_FOLDER,i))

In [20]:
!python run_inference.py



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /ssd_Samsung870_2T/docker_submission/nnUNet/nnunet/nnUNet_trained_models/nnUNet/3d_fullres/Task119_final_model/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 8 unique case ids, here are some examples: ['crossmoda2022_etz_218' 'crossmoda2021_ldn_241' 'crossmoda2021_ldn_241'
 'crossmoda2022_etz_241' 'crossmoda2022_etz_241' 'crossmoda2021_ldn_229'
 'crossmoda2022_etz_218' 'crossmoda2022_etz_241']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 8
number of cases that still need to be predicted: 8